In [37]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

Creating the master Dataframe -->

In [38]:
columns = ['Postal Code','Borough','Neighborhood']
Master_DF = pd.DataFrame(columns=columns)

I'll be using Beautiful Soup to parse the HTML content of the webpage

In [39]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
data = BeautifulSoup(response.text,'html.parser')

In [40]:
table = data.find_all('table')[0]
rows = table.find_all('tr')
print(len(rows))
print(rows[0])

181
<tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>


After inspecting the webpage, the required table is contained within the table tag, which I've extracted using the find_all command

Within the table , the individual rows are contained within the tr tag. The length of rows is 181 which means we have 180 records. The first record doesn't contin the information, it's just the name of columns

In [41]:
for x in rows[1:]:
    temp = x.find_all('td')
    list_temp = []
    for i in temp:
        list_temp.append(str(i.text.strip()))
    #print(list_temp)
    pc = list_temp[0]
    b = list_temp[1]
    n = list_temp[2]
    Master_DF = Master_DF.append({'Postal Code':pc,'Borough':b,'Neighborhood':n},ignore_index=True)

In [42]:
Master_DF.shape

(180, 3)

Iterating over the rows , the individual elements are contained within the <td> tag which I am accesing and appending to the Master Dataframe

In [43]:
Master_DF = Master_DF[Master_DF['Borough'] != 'Not assigned'].reset_index()
Master_DF.shape

(103, 4)

In [44]:
lat_lon_csv = pd.read_csv('Geospatial_Coordinates.csv')

In [45]:
Master_DF = Master_DF.merge(lat_lon_csv,on='Postal Code',how='inner').drop('index',axis=1)

In [46]:
Master_DF.shape

(103, 5)

In [47]:
Master_DF.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
